In [2]:
! pip install smartmoneyconcepts

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   -- ------------------------------------- 0.8/14.9 MB 5.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/14.9 MB 5.0 MB/s eta 0:00:03
   ------- -------------------------------- 2.9/14.9 MB 4.9 MB/s eta 0:00:03
   ----------- ---------------------------- 4.2/14.9 MB 6.3 MB/s eta 0:00:02
   ----------- ---------------------------- 4.2/14.9 MB 6.3 MB/s eta 0:00:02
   ------------------- -------------------- 7.1/14.9 MB 5.7 MB/s eta 0:00:02
   --------------------------------- ------ 12.6/14.9 MB 8.7 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 9.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ------------ --------------------------- 3.4/10.8 MB 16.7 MB/s eta 0:00:01
   ---------------------------------------- 10.8/10.8 MB 28.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Unins

  You can safely remove it manually.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import MetaTrader5 as mt5
from smartmoneyconcepts import smc

In [2]:
import pytz
from datetime import datetime
import pandas as pd


timezone = pytz.timezone("Etc/UTC")

if not mt5.initialize():
    raise Exception("initialize() failed, error code =", mt5.last_error())

date_from_get_data = datetime(2021, 10, 1, tzinfo=timezone)
date_to_get_data = datetime(2024, 9, 1, tzinfo=timezone)

# Obtener las tasas históricas
rates = mt5.copy_rates_range('EURUSD', mt5.TIMEFRAME_H4, date_from_get_data, date_to_get_data)

# Crear DataFrame con las tasas
df = pd.DataFrame(rates)

# Convertir el tiempo de segundos a formato datetime
df['time'] = pd.to_datetime(df['time'], unit='s')

# Renombrar columnas para el ticker principal
df = df.rename(columns={
    'time': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'tick_volume': 'Volume'
}).set_index('Date')

df.index = df.index.tz_localize('UTC').tz_convert('UTC')

mt5.shutdown()

df

,Open,High,Low,Close,Volume,spread,real_volume
Date,,,,,,,
2021-10-01 00:00:00+00:00,1.15740,1.15819,1.15652,1.15652,3704,0,0
2021-10-01 04:00:00+00:00,1.15652,1.15821,1.15630,1.15806,7291,0,0
2021-10-01 08:00:00+00:00,1.15806,1.15912,1.15745,1.15786,14990,0,0
2021-10-01 12:00:00+00:00,1.15786,1.16070,1.15725,1.16038,14825,0,0
2021-10-01 16:00:00+00:00,1.16038,1.16075,1.15838,1.15968,21509,0,0
...,...,...,...,...,...,...,...
2024-08-30 04:00:00+00:00,1.10805,1.10821,1.10697,1.10740,6224,2,0
2024-08-30 08:00:00+00:00,1.10740,1.10951,1.10735,1.10749,14795,2,0
2024-08-30 12:00:00+00:00,1.10748,1.10882,1.10577,1.10679,19775,2,0


In [3]:
fvg = smc.fvg(df, join_consecutive=False)
fvg

,FVG,Top,Bottom,MitigatedIndex
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
4543,NaN,NaN,NaN,NaN
4544,NaN,NaN,NaN,NaN
4545,NaN,NaN,NaN,NaN
4546,-1.0,1.10577,1.10567,0.0


In [4]:
fvg = fvg[~fvg['FVG'].isna()]
fvg

,FVG,Top,Bottom,MitigatedIndex
8,1.0,1.16079,1.16040,13.0
12,-1.0,1.16161,1.16134,56.0
13,-1.0,1.16105,1.16051,15.0
19,-1.0,1.15920,1.15901,53.0
20,-1.0,1.15833,1.15542,22.0
...,...,...,...,...
4531,-1.0,1.11744,1.11607,0.0
4532,-1.0,1.11582,1.11456,0.0
4536,1.0,1.11272,1.11200,4538.0
4538,-1.0,1.11272,1.11000,0.0


In [7]:
fvg[fvg['Top'] > fvg['Bottom']]

,FVG,Top,Bottom,MitigatedIndex
8,1.0,1.16079,1.16040,13.0
12,-1.0,1.16161,1.16134,56.0
13,-1.0,1.16105,1.16051,15.0
19,-1.0,1.15920,1.15901,53.0
20,-1.0,1.15833,1.15542,22.0
...,...,...,...,...
4531,-1.0,1.11744,1.11607,0.0
4532,-1.0,1.11582,1.11456,0.0
4536,1.0,1.11272,1.11200,4538.0
4538,-1.0,1.11272,1.11000,0.0


In [23]:
swing_highs_lows = smc.swing_highs_lows(df, swing_length = 10)
swing_highs_lows[~swing_highs_lows['HighLow'].isna()]

,HighLow,Level
0,1.0,1.15819
21,-1.0,1.15292
38,1.0,1.15867
47,-1.0,1.15242
57,1.0,1.16245
...,...,...
4480,-1.0,1.09495
4505,1.0,1.11740
4510,-1.0,1.10980
4518,1.0,1.12016


In [ ]:

bos_choch = smc.bos_choch(df, swing_highs_lows, close_break = True)
bos_choch


,BOS,CHOCH,Level,BrokenIndex
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
4543,NaN,NaN,NaN,NaN
4544,NaN,NaN,NaN,NaN
4545,NaN,NaN,NaN,NaN
4546,NaN,NaN,NaN,NaN


In [15]:
bos_choch[~bos_choch['BOS'].isna()]

,BOS,CHOCH,Level,BrokenIndex
57,1.0,NaN,1.16245,72.0
124,-1.0,NaN,1.15353,153.0
153,-1.0,NaN,1.15134,172.0
199,-1.0,NaN,1.12632,220.0
363,1.0,NaN,1.13435,383.0
...,...,...,...,...
4312,1.0,NaN,1.08450,4328.0
4382,-1.0,NaN,1.08256,4401.0
4431,1.0,NaN,1.10089,4472.0
4474,1.0,NaN,1.10474,4492.0


In [16]:
bos_choch[~bos_choch['CHOCH'].isna()]

,BOS,CHOCH,Level,BrokenIndex
21,NaN,-1.0,1.15292,46.0
38,NaN,1.0,1.15867,53.0
68,NaN,-1.0,1.15717,124.0
293,NaN,1.0,1.13549,327.0
476,NaN,1.0,1.13689,538.0
554,NaN,-1.0,1.13963,571.0
751,NaN,1.0,1.10379,765.0
758,NaN,-1.0,1.09448,796.0
838,NaN,-1.0,1.07576,878.0
911,NaN,-1.0,1.04903,956.0


In [17]:
ob = smc.ob(df, swing_highs_lows, close_mitigation = False)
ob


,OB,Top,Bottom,OBVolume,MitigatedIndex,Percentage
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
4543,NaN,NaN,NaN,NaN,NaN,NaN
4544,NaN,NaN,NaN,NaN,NaN,NaN
4545,NaN,NaN,NaN,NaN,NaN,NaN
4546,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
ob[~ob['OB'].isna()]

,OB,Top,Bottom,OBVolume,MitigatedIndex,Percentage
118,-1.0,1.16920,1.16243,60976.0,0.0,34.0
144,-1.0,1.16162,1.16030,39686.0,0.0,15.0
166,-1.0,1.16086,1.15714,57537.0,0.0,32.0
568,-1.0,1.14948,1.13749,14912.0,0.0,87.0
593,-1.0,1.13958,1.13703,26082.0,0.0,27.0
1373,-1.0,1.02028,1.01630,40543.0,1739.0,42.0
1653,1.0,0.97775,0.97047,65729.0,0.0,47.0
1707,1.0,0.97704,0.97301,55334.0,0.0,21.0
1822,1.0,1.03995,1.02902,86499.0,0.0,42.0
1850,1.0,1.04842,1.04431,51944.0,0.0,19.0


In [21]:
liq = smc.liquidity(df, swing_highs_lows, range_percent = 0.01)
liq[~liq['Liquidity'].isna()]


,Liquidity,Level,End,Swept
21,-1.0,1.152552,153.0,172.0
144,1.0,1.161240,166.0,0.0
209,1.0,1.137050,425.0,441.0
285,-1.0,1.122773,336.0,506.0
380,-1.0,1.127693,430.0,503.0
451,1.0,1.148713,568.0,0.0
512,-1.0,1.111395,628.0,646.0
635,1.0,1.127485,2798.0,0.0
687,1.0,1.112925,719.0,770.0
734,-1.0,1.095285,758.0,796.0


In [11]:
smc.sessions(df, 'New York', time_zone = "UTC")


,Active,High,Low
0,0,0.00000,0.00000
1,0,0.00000,0.00000
2,0,0.00000,0.00000
3,0,0.00000,0.00000
4,1,1.16075,1.15838
...,...,...,...
4543,0,0.00000,0.00000
4544,0,0.00000,0.00000
4545,0,0.00000,0.00000
4546,1,1.10801,1.10438
